In [1]:
import ipywidgets as widgets
widgets.IntSlider()


IntSlider(value=0)

In [3]:
import gradio as gr
gr.Interface(lambda x: f"Hello {x}!", gr.Textbox(), gr.Textbox()).launch()


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [9]:
import gradio as gr
import ollama
import time  # For simulating status updates

# Store conversation history
conversation_history = []

# Function to generate the initial itinerary with a loading message
def generate_itinerary(country, days, activities, status):
    global conversation_history  
    conversation_history = []  # Reset history for new itinerary

    status = "⏳ Generating your itinerary... Please wait."  # Update status
    time.sleep(1)  # Simulate brief delay for better UX

    prompt = f"Plan a {days}-day trip to {country}, including these activities: {activities}. Keep it detailed yet concise."
    
    response = ollama.chat(model="llama3.2:1b", messages=[{"role": "user", "content": prompt}])
    
    itinerary = response['message']['content']

    # Add response to conversation history
    conversation_history.append({"role": "user", "content": prompt})
    conversation_history.append({"role": "assistant", "content": itinerary})

    return itinerary, "✅ Itinerary ready! You can refine it below."  # Update status when done

# Function to refine the itinerary through chat
def chat_with_ollama(user_input, status):
    global conversation_history

    status = "💬 Thinking... Please wait."  # Show status message while AI processes
    
    # Add user input to conversation history
    conversation_history.append({"role": "user", "content": user_input})

    # Generate response
    response = ollama.chat(model="llama3.2:1b", messages=conversation_history)
    
    # Save assistant's response
    conversation_history.append({"role": "assistant", "content": response['message']['content']})

    # Format chat history for display
    formatted_history = "\n\n".join([f"**{msg['role'].capitalize()}**: {msg['content']}" for msg in conversation_history])
    
    return formatted_history, "✅ Response ready!"  # Update status

# Reset function to clear chat history
def reset_chat():
    global conversation_history
    conversation_history = []
    return "Chat reset. Start a new itinerary!", "🔄 Chat reset. Ready for a new plan!"

# Create the Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🏝️ AI Travel Itinerary Planner (Interactive)")
    
    with gr.Row():
        country = gr.Textbox(label="Destination Country", placeholder="e.g., Sweden")
        days = gr.Number(label="Number of Days", value=3)
    
    activities = gr.Textbox(label="Activities (comma-separated)", placeholder="e.g., nature walks, cycling, food tours")

    generate_btn = gr.Button("✈️ Generate Initial Itinerary")
    
    chatbox = gr.Textbox(label="Chat with AI", placeholder="Ask follow-up questions...", lines=2)
    chat_output = gr.Markdown(label="Conversation History")  

    refine_btn = gr.Button("💬 Ask AI")
    reset_btn = gr.Button("🔄 Reset Chat")

    # Status message stored in gr.State()
    status = gr.State("ℹ️ Click 'Generate Itinerary' to start.")
    status_display = gr.Markdown(status.value)

    # Connect functions to buttons
    generate_btn.click(fn=generate_itinerary, inputs=[country, days, activities, status], outputs=[chat_output, status_display])
    refine_btn.click(fn=chat_with_ollama, inputs=[chatbox, status], outputs=[chat_output, status_display])
    reset_btn.click(fn=reset_chat, outputs=[chat_output, status_display])

demo.launch()








* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
